<a href="https://colab.research.google.com/github/Valancinis/WranglingData/blob/main/OORWrangle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
import requests, io, pandas as pd
from datetime import datetime, timedelta
from google.colab import drive, files

#_Functions_

#Exporting csv to google drive
   #Mounting drive. Need to do this once
def mountingDrive():
    drive.mount('/content/drive')

  #Exporting
def exportingOor():
   #where to save path
  path = '/content/drive/My Drive/Python/output.csv'
  with open(path, 'w', encoding = 'utf-8-sig') as f:
    oordf.to_csv(f)
    print("File exported")

  #Exporting
def exportingOpenPOs():
  #where to save path
  path = '/content/drive/My Drive/Python/openpos.csv'
  with open(path, 'w', encoding = 'utf-8-sig') as f:
    navdf.to_csv(f)
    print("File exported")

      #Exporting
def exportingFinal():
  #where to save path
  #path = '/content/drive/My Drive/Python/final.csv'
  #with open(path, 'w') as f:
    #mergeddf.to_excel(f)
  mergeddf.to_excel(r'/content/drive/My Drive/Python/final.xlsx', index = False)
  print("File exported")

def downloadExcel():
  mergeddf.to_excel('output excel.xlsx')
  files.download('output excel.xlsx')

#_End of Functions_

  #reachout out to github repository
github_session = requests.Session()

    # providing raw url to download csv/excel from github
#csv_url = 'https://raw.githubusercontent.com/Valancinis/WranglingData/main/OOR for Thermo Fisher Scientific - LT.csv'
excelUrlOOR = 'https://raw.githubusercontent.com/Valancinis/WranglingData/main/OOR for Thermo Fisher Scientific - LT.xlsx'
excel_url = 'https://raw.githubusercontent.com/Valancinis/WranglingData/main/LED OOR Query.xlsx'

download1 = github_session.get(excelUrlOOR).content
download2 = github_session.get(excel_url).content
#importdataOOR = pd.read_csv(io.StringIO(download1.decode('utf-8')), on_bad_lines='skip', sep = ";", keep_default_na=False)

importdataOOR = pd.read_excel(download1, na_filter=False)
importdataOPO = pd.read_excel(download2)

   #picking needed data
oordf = importdataOOR.iloc[:,[2,10,16]].copy()
navdf = importdataOPO.iloc[:,[0,1,3]].copy()

oordf.rename({"Material": "ID", "Current ship date": "Delivery date"}, axis='columns', inplace=True)

   #setting 'Delivery date' column datatype to datetime
oordf['Delivery date'] = pd.to_datetime(oordf['Delivery date'], errors='coerce')
navdf['Expected Receipt Date'] = pd.to_datetime(navdf['Expected Receipt Date'], errors='coerce')

for i, row in oordf.iterrows():
  oordf.at[i,'Delivery date'] = (oordf.at[i,'Delivery date'] + timedelta(days=8))

navdf['Ddelta'] = (navdf['Expected Receipt Date'] - oordf['Delivery date'])/timedelta(days=1)

  #merge 2 dataframes
mergeddf = pd.merge(oordf, navdf)
  #Add a new column
mergeddf['Update'] = ""

  #removing non needed column
del mergeddf['Expected Receipt Date']

  #looping through the dataframe, turning timedelta objects to int and updating "Update" column values if row needs to be updated in the ERP system

for i in mergeddf.index:
  n = mergeddf.loc[i, 'Ddelta']
  if (n > 9) or (n < -9):
    mergeddf.loc[i,'Update'] = "True"
  else:
    mergeddf.loc[i,'Update'] = "False"

print(mergeddf.head())

#mountingDrive()
  #_Exporting files
#exportingOor()
#exportingOpenPOs()
#exportingFinal()
#downloadExcel()


            PO           ID Delivery date    Vendor ID  Ddelta Update
0  PO-20-12297      916-021    2022-07-07      916-021   -42.0   True
1  PO-20-13087    5000-1012    2022-09-19    5000-1012   -79.0   True
2  PO-21-02186  342020-0125    2023-03-23  342020-0125  -293.0   True
3  PO-21-03547      AB-2005    2022-08-14      AB-2005   -59.0   True
4  PO-21-04128       3465NK    2022-09-08       3465NK     0.0  False


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>